<a href="https://colab.research.google.com/github/DongyounKim/Classification/blob/main/Vis_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!pip install datasets
!pip install transformers
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
data_dir = '/content/gdrive/MyDrive/Corrosion Condition State Classification/original'

In [ ]:
# option 2: local folder
from datasets import load_dataset
#dataset = load_dataset('jleeeee/bridge_corrosion_classification')
dataset = load_dataset("imagefolder", data_dir=data_dir)

Resolving data files:   0%|          | 0/1320 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/440 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#k = load_dataset('Dodon/Corrosion')
#k['train']['image']

In [ ]:
example = dataset['train'].select(range(10))

In [ ]:
example.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['images', 'masks'], id=None)}

In [ ]:
print(example[0]['label'])

0


#preprocessing

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
# size = (
#     image_processor.size["shortest_edge"]
#     if "shortest_edge" in image_processor.size
#     else (image_processor.size["height"], image_processor.size["width"])
# )
# _transforms = Compose([RandomResizedCrop(size), ToTensor()])

In [ ]:
import json
import cv2
import torch
from glob import glob
import os
from PIL import Image
import numpy as np


from pathlib import Path

from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)


class MyDataset():
    def __init__(self, img_dir):
      self.dir = img_dir
      self.img_files = glob(os.path.join(img_dir, 'images/*'))
      self.lab_files = glob(os.path.join(img_dir, 'json/*'))

    def __len__(self):
      return len(self.img_files)

    # def __getitem__(self, idx):
    #   img_path = self.img_files[idx]
    #   json_path = self.lab_files[idx]

    #   image = cv2.imread(img_path)

    #   f = open(json_path)
    #   json_file = json.load(f)
    #   label = np.max([int(a['label'][:1]) for a in json_file['shapes']])
    #   f.close()

    #   # B,C,H,W
    #   # HWC -> CHW
    #   # print(image.shape)
    #   image_ra = np.transpose(image, axes=[2, 0, 1])
    #   label_ra = label

    #   return torch.from_numpy(image_ra).float(), torch.from_numpy(label_ra).float()

    def __call__(self, example = True):


      batch = {}
      # batch_size = len(features)
      _label = []
      _image = []
      for _idx, _ele in enumerate(self.img_files):
        _img_name = os.path.basename(_ele)
        _img_name = os.path.splitext(_img_name)[0]

        label_path = os.path.join(self.dir, 'json/{}.json'.format(_img_name))
        #print("label_path", label_path)


        im = Image.open(_ele)
        #print(im)
        #print(_ele)
        #batch["image"] =
        _image.append(im)
        #torch.stack([transformer(im)])
        #inputs = torch.stack([self.processor(im, return_tensors='pt')])
        #print(inputs)
        f = open(label_path)
        json_file = json.load(f)

        _label.append([np.max([int(a['label'][:1]) for a in json_file['shapes']])])
        #_label.append()
        #print(_label)
        #batch["image"] = torch.stack(_label)

        batch.update({"image": _image})
        batch.update({"label": _label})
        #batch.update({"label": np.max([int(a['label'][:1]) for a in json_file['shapes']])})
        #print(transformer(im))
      return batch

In [ ]:
_ex = MyDataset(os.path.join(data_dir,'Test/'))
batch = _ex()

In [ ]:
print(batch['image'][1])
print(batch['label'])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x1200 at 0x7C1C91C93C10>
[[3], [3], [4], [3], [3], [3], [3], [3], [4], [4], [4], [3], [3], [4], [3], [3], [4], [2], [2], [2], [2], [3], [3], [3], [3], [3], [4], [3], [3], [4], [3], [3], [3], [3], [3], [4], [3], [3], [4], [4], [3], [3], [3], [4]]


In [ ]:
processor(batch['image'], return_tensors='pt')

{'pixel_values': tensor([[[[-0.2549, -0.2549, -0.2706,  ...,  0.3176,  0.3255,  0.3333],
          [-0.2549, -0.2471, -0.2627,  ...,  0.3255,  0.3176,  0.3020],
          [-0.2627, -0.2549, -0.2549,  ...,  0.3412,  0.3255,  0.3412],
          ...,
          [ 1.0000,  1.0000,  1.0000,  ..., -0.6549, -0.7098, -0.7176],
          [ 1.0000,  1.0000,  1.0000,  ..., -0.6235, -0.6549, -0.7333],
          [ 1.0000,  1.0000,  1.0000,  ..., -0.6941, -0.6078, -0.6157]],

         [[-0.3333, -0.3333, -0.3490,  ...,  0.3255,  0.3020,  0.2941],
          [-0.3333, -0.3255, -0.3412,  ...,  0.3098,  0.2863,  0.2549],
          [-0.3412, -0.3333, -0.3333,  ...,  0.2941,  0.2784,  0.3020],
          ...,
          [ 1.0000,  1.0000,  1.0000,  ..., -0.7647, -0.7882, -0.7804],
          [ 1.0000,  1.0000,  1.0000,  ..., -0.7333, -0.7490, -0.8196],
          [ 1.0000,  1.0000,  1.0000,  ..., -0.7725, -0.7098, -0.7412]],

         [[-0.4118, -0.4118, -0.4275,  ...,  0.2314,  0.1529,  0.0902],
          [-0

In [ ]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [ ]:
import numpy as np
from datasets import load_metric

# metrics
metric = load_metric('accuracy')
def compute_metrics(p):
    labels = p.label_ids
    preds = p.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'f1': f1
    }

# metric = load_metric("accuracy")
# def compute_metrics(p):
#     return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-16-487d1bf6bbe0>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
import torch

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")

Training on GPU


In [ ]:
from transformers import ViTForImageClassification
labels = ['fair', 'poor', 'severe']
id2label={}
label2id={}
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
model = model.to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# from transformers import ViTForImageClassification

# #labels = ds['train'].features['labels'].names
# class_names = ['fair', 'poor', 'severe']
# labels = [2,3,4]
# model = ViTForImageClassification.from_pretrained(
#     model_name_or_path,
#     num_labels=len(labels),
#     #id2label={str(i): c for i, c in enumerate(labels)},
#     #label2id={c: str(i) for i, c in enumerate(labels)}
# )

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

ImportError: ignored

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=processor,
)

NameError: ignored

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'beans',
    "tags": ['image-classification'],
}

# if training_args.push_to_hub:
#     trainer.push_to_hub('🍻 cheers', **kwargs)
# else:
#     trainer.create_model_card(**kwargs)

In [ ]:
def transform(example_batch, img_dir = 'k'):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = img_dir
    return inputs

In [ ]:
prepared_ds = example.with_transform(transform)

In [ ]:
example

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

In [ ]:
#Dataset
# JEPG, LABEL ~ path
#1. Pre-processing ~ batch [Input format]
## 1. JEPG -> Tensor
## 2. Label -> integer
## ~ Dict(Label: [1,2,..], image: [tensor])

## Vis-Transform

AttributeError: ignored

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Exception ignored in: <function _xla_gc_callback at 0x7a4f27ce2170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
